# Problem 3 Sample Code

This sample code is meant as a guide on how to use PyTorch and how to use the relevant model layers. This not a guide on how to design a network and the network in this example is intentionally designed to have poor performace.

In [221]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, Dataset, DataLoader
import seaborn as sns
from sklearn.model_selection import train_test_split

In [222]:
df = pd.read_csv('out.csv')

df = df.replace('',np.nan)
df = df.dropna(axis="rows", how="any")
df = df.drop(columns= "uid")

df_X = df.drop(columns= "label")
for col in df_X:
    df_X[col] = [(element - np.mean(df_X[col])) / np.std(df_X[col]) for element in df_X[col]]
print("===dataframe=== \n {}".format(df))
X = np.array(df_X)
y = np.array(df["label"])

# def normalize(x):
#     x = np.asarray(x)
#     return (x - x.min()) / (np.ptp(x))
# X_normed = []
# for i in range(0, len(X)):
#     X_normed.append(X[i])
# # X = (X - X.mean())/(X.std())
# print(" - NORMALIZED  X \n {} - ".format(X))


train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.01)
# print(" pre TRAIN X \n {}".format(train_X))

train_X = torch.tensor(train_X, dtype=torch.float32)
# print("TENSOR  X \n {}".format(train_X))
train_y = torch.tensor(train_y, dtype=torch.long)
# 80/20 x values as floats
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.long)
print("test_x torch tensor {}".format(test_X))
print("test_x torch tensor shape {}".format(test_X.shape))

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True) 

print ("x \n{}".format(df))
print ("y \n{}".format(df))

# x_train = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_train = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# x_test = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_test = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# train_dataset = TensorDataset(x_train, y_train)
# test_dataset = TensorDataset(x_test, y_test)

===dataframe=== 
        label     squig   lin_reg  lin_reg_chi  stddev_step_speed
0          0  0.213184  0.992533     0.988869          17.360221
1          0  0.208286  0.973844     1.938908          17.308306
2          0  0.217461  0.894242     6.604954          17.300169
3          0  0.215804  0.967026     5.758685          17.376369
4          0  0.195877  0.000296     1.512429          17.205146
...      ...       ...       ...          ...                ...
16075      0  0.444782  0.999245     0.867075          18.036484
16076      0  0.381438  0.999707     1.345129          26.151765
16077      0  0.436170  0.998892     3.587700          23.801682
16078      0  0.471862  0.998953     0.680709          16.319528
16079      0  0.385406  0.999056     7.286809          30.031311

[16013 rows x 5 columns]
test_x torch tensor tensor([[ 1.2461e+00, -2.0341e+00,  3.4708e+00,  3.8806e-01],
        [ 9.7287e-01, -6.9645e-01,  1.1687e+00,  1.0550e+00],
        [-5.6451e-01,  5.8660e-0

We can convert images to numpy arrays and plot them with matplotlib:

## Network Definition
Let's instantiate a model and take a look at the layers.

In [223]:
model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2),
    nn.Sigmoid()
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()

torch.manual_seed(15552494823729223621)

## Training
We also choose an optimizer and a loss function.

We could write our training procedure manually and directly index the `Dataset` objects, but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:

We now write our backpropagation loop, training for 10 epochs.

In [224]:
model.train()

for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        # print("data \n{}".format(data))
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        # print(" -- OPUTPUT -- \n{}".format(output))
        # output = output.flatten()

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: 0.6720
Train Epoch: 2  Loss: 0.6337
Train Epoch: 3  Loss: 0.5476
Train Epoch: 4  Loss: 0.4762
Train Epoch: 5  Loss: 0.4491
Train Epoch: 6  Loss: 0.3985
Train Epoch: 7  Loss: 0.3881
Train Epoch: 8  Loss: 0.3853


## Testing
We can perform forward passes through the network without saving gradients.

In [225]:
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        print(output)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        print(pred)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy : %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

tensor([[9.2670e-01, 4.2178e-02],
        [8.7577e-04, 9.9581e-01],
        [1.6485e-02, 9.6606e-01],
        [9.3345e-01, 3.7218e-02],
        [9.3148e-01, 3.8631e-02],
        [6.4950e-01, 2.8858e-01],
        [5.1684e-02, 9.2547e-01],
        [2.0734e-02, 9.6042e-01],
        [1.6500e-01, 8.2898e-01],
        [4.8770e-03, 9.8575e-01],
        [9.4076e-01, 3.1720e-02],
        [8.6499e-01, 8.6637e-02],
        [8.8084e-01, 7.4615e-02],
        [4.3783e-01, 5.2895e-01],
        [9.3678e-01, 3.4313e-02],
        [5.5446e-01, 3.9355e-01],
        [4.4605e-02, 9.3185e-01],
        [8.2677e-01, 9.1609e-02],
        [1.6282e-02, 9.6685e-01],
        [9.3769e-01, 3.3900e-02],
        [1.7946e-01, 8.1886e-01],
        [6.0973e-03, 9.8340e-01],
        [9.4065e-01, 3.1549e-02],
        [9.4360e-01, 2.9402e-02],
        [9.3638e-01, 3.4817e-02],
        [9.4448e-01, 2.8994e-02],
        [9.3884e-01, 3.3081e-02],
        [9.3380e-01, 3.6879e-02],
        [9.2522e-01, 4.2871e-02],
        [1.232

In [226]:
#   REAL TESTING
test = pd.read_csv('test.csv')

df_vals = test.drop(columns= "uid")
for col in df_vals:
    df_vals[col] = [(element - np.mean(df_vals[col])) / np.std(df_vals[col]) for element in df_vals[col]]
print("===dataframe=== \n {}".format(test))

test_X = np.array(df_vals)
test_X = torch.tensor(test_X, dtype=torch.float32)

print(test_X)
print(test_X.shape)



===dataframe=== 
            uid     squig   lin_reg  lin_reg_chi  stddev_step_speed
0     lab_19_0  0.228740  0.878979     0.267350          19.661653
1     lab_19_1  1.056858  0.072402   407.576553          18.135286
2     lab_19_2  0.462892  0.998963    27.825655          86.806286
3     lab_19_3  0.653292  0.991160   170.673846          81.682927
4     lab_19_4  0.941941  0.987416     1.985455          10.228939
..         ...       ...       ...          ...                ...
472   lab_42_9  0.645642  0.989693   549.380346          85.433759
473  lab_42_10  0.528179  0.926821   244.652959          32.603626
474  lab_42_11  0.489326  0.952655   132.548405          17.927726
475  lab_42_12  0.344979  0.999726     1.653985          36.955453
476  lab_42_13  0.374670  0.997562    18.285068          46.364285

[477 rows x 5 columns]
tensor([[-7.3850e-01, -1.1713e-02, -2.0709e-01, -6.0311e-01],
        [ 1.8809e+00, -3.3169e+00,  2.0968e-01, -6.7312e-01],
        [ 2.1488e-03,  4.7996e

In [227]:
output = model(test_X)
#print(output)
pred = output.argmax(dim=1, keepdim=True)

pred = np.array(pred)
print(pred)
submission = pd.DataFrame(columns=['UID', 'label'])
submission['UID'] = test['uid']
submission['label'] = pred
print(submission)

submission.to_csv('submission.csv', index=False)

[[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
